### 场景 

根据时间打印日志。（当为指定时间是按照函数调用时间来显示时间。）（参数是变化的 not mutable类型）

In [12]:
from datetime import datetime
from time import sleep

def log(message, when = datetime.now()):
    print('%s: %s' % (when, message))
    
log('Hi there!')
sleep(1)
log('Hi again!')

2016-11-24 14:37:17.714388: Hi there!
2016-11-24 14:37:17.714388: Hi again!


此处始终只显示函数加载时的时间。因为datetime.now()只在模块加载时执行。再次调用时不会重新执行。

### 解决方案

Python中动态默认值，习惯上把默认值设置为None，并在文档字符串（docstring）里把None所对应的时间行为描述出来（参见49条）。编写函数代码是，如果发现该参数是None，就会将其设为实际的默认值。

In [13]:
def log(message, when=None):
    """Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time
    """
    when = datetime.now() if when is None else when
    print('%s: %s' % (when, message))
    

In [14]:
log('Hi there!')
sleep(1)
log('Hi again!')

2016-11-24 14:44:48.172589: Hi there!
2016-11-24 14:44:49.172716: Hi again!


（其实不需要使用默认参数）

### 场景2--参数默认值为mutable可变类型 

按照JSON格式解析字符串。若失败，则默认返回空字典。

In [17]:
import json
def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)


Foo: {'stuff': 5, 'meep': 1}
Bar: {'stuff': 5, 'meep': 1}


预期结果为foo和bar里面各有一对不同的键和值。而结果显示，修改其中一个后，另一个似乎也受到了影响。这种错误的**根本原因是**：foo和bar都是default参数默认值的那个字典对象。

In [18]:
assert foo is bar

### 解决办法

把关键字的默认值设为None，并在函数的文档字符串中描述他的实际行为。

In [19]:
def decode(data, default = None):
    """Load JSON data from a string.
    
    Args:
        data: JSON data to decode.
        default: Value to return if decoding fails.
            Defaults to an empty dictionary.
    """
    if default is None:
        default = {}
    try:
        return json.loads(data)
    except ValueError:
        return default

In [20]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)

Foo: {'stuff': 5}
Bar: {'meep': 1}


### 要点 

1. 参数默认值， 之后在程序加载模块并调用该函数时赋值一次。对于{}或[]等mutable类型，这可能大致奇怪的行为。
2. 对于动态值作为实际默认值的关键字参数来说，应该把形式上的默认值为None， 并在函数的文档字符串里面描述该默认值所对应的实际行为。